<a href="https://colab.research.google.com/github/JuanchoGithub/mejuri-data-challenge/blob/master/read_from_api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Imports
import pandas as pd
import numpy as np
import requests
import json
from google.cloud import bigquery
from requests.auth import HTTPBasicAuth

In [0]:
#Variables

#https://retailnext.atlassian.net/wiki/spaces/KB/pages/145129473/API

#Used this to convert the KB sample curl commands to valid python
#https://curl.trillworks.com/#python


toronto = '8c1d0d0a-9fdd-11e8-8003-0000e6e67141'
newyork = '9282ba78-ee2f-11e8-9722-80007e030792'
losangeles = '65c1be80-ac93-11e9-9a41-c0005e4bb78f'
sanfrancisco = 'c3adc3bc-1a8f-11ea-955a-c000ff5d2ae7'

name = "mejuri"
accesskey = "3b48b028-780d-11ea-9264-80007aa91c1c"
secretkey = "RFFFMrBwOzSmawYhijB5-j4f7ec"

url = "https://" + name + ".api.retailnext.net/v1/datamine"
#header = {"content-type": "application/json"}

#url = 'https://mejuri.api.retailnext.net/v1/datamine'

req = {}
locations = {"toronto":toronto, "newyork":newyork, "losangeles":losangeles, "sanfrancisco":sanfrancisco}
req['time_ranges'] = [{"type":"store_hours"}]
req["group_bys"] = [{"value":1, "unit":"minutes", "group":"time"}]
req["date_ranges"] = [{'first_day': '2019-12-09', 'last_day': '2019-12-09'}]
req["metrics"] = ["traffic_in", "traffic_out"]
headers = {'content-type': 'application/json', 'Accept-Charset': 'UTF-8'}


In [4]:
datos = []
for location, value in locations.items():
  req['locations'] = [value]
  json_req = json.dumps(req)
  r = requests.post(url,data=json_req, auth=HTTPBasicAuth(accesskey, secretkey)).json()

  for metric in r["metrics"]:
    for data in metric["data"]:
      group = data["group"]
      datos.append({"store":location, "metric":metric["name"], "value":data["value"], "start_time":group["finish"], "end_time":group["start"]})

df = pd.DataFrame.from_records(datos)
df

,store,metric,value,start_time,end_time
0,toronto,traffic_in,8,12:01,12:00
1,toronto,traffic_in,7,12:16,12:15
2,toronto,traffic_in,3,12:31,12:30
3,toronto,traffic_in,4,12:46,12:45
4,toronto,traffic_in,2,13:01,13:00
...,...,...,...,...,...
507,sanfrancisco,traffic_out,0,22:46,22:45
508,sanfrancisco,traffic_out,0,23:01,23:00
509,sanfrancisco,traffic_out,0,23:16,23:15
510,sanfrancisco,traffic_out,0,23:31,23:30


In [6]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gc_service_key.json'

project_id = 'idyllic-pact-273015'

client = bigquery.Client(project=project_id)

table_id = 'mejuri_data_challenge.traffic'
# Since string columns use the "object" dtype, pass in a (partial) schema
# to ensure the correct BigQuery data type.
job_config = bigquery.LoadJobConfig(schema=[
    bigquery.SchemaField("store", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("metric", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("value", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("start_time", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("end_time", "STRING", mode="REQUIRED")
])

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)

# Wait for the load job to complete.
job.result()

In [8]:
%%bigquery
SELECT * FROM `idyllic-pact-273015.mejuri_data_challenge.traffic` LIMIT 1000

,store,metric,value,start_time,end_time
0,losangeles,traffic_in,0,00:01,00:00
1,losangeles,traffic_out,0,00:01,00:00
2,sanfrancisco,traffic_in,0,00:01,00:00
3,sanfrancisco,traffic_out,0,00:01,00:00
4,losangeles,traffic_in,0,00:16,00:15
...,...,...,...,...,...
995,sanfrancisco,traffic_out,0,22:01,22:00
996,losangeles,traffic_in,0,22:16,22:15
997,losangeles,traffic_out,0,22:16,22:15
998,sanfrancisco,traffic_in,0,22:16,22:15
